In [1]:
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
print("Done!")

Done!


In [2]:
dataframe = pd.read_csv("housing.csv", sep=',', header=0)
dataset = dataframe.values
dataframe.head()

,BIZPROP,ROOMS,AGE,HIGHWAYS,TAX,PTRATIO,LSTAT,VALUE
0,19.58,7.489,90.8,5,403,14.7,1.73,50.0
1,19.58,7.802,98.2,5,403,14.7,1.92,50.0
2,19.58,8.375,93.9,5,403,14.7,3.32,50.0
3,19.58,7.929,96.2,5,403,14.7,3.70,50.0
4,2.46,7.831,53.6,3,193,17.8,4.45,50.0


In [3]:
features = dataset[:,0:7]
target = dataset[:,7]

In [4]:
#fix random seed for reproducibility
seed = 9
numpy.random.seed(seed)

In [5]:
def simple_shallow_seq_net():
    #create a sequential ANN
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal',activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [6]:
seed = 9 
kfold = KFold(n_splits=10, random_state=seed) #randomly divide data into 10 subsets for training and validation


C:\Users\Asus\Anaconda3\envs\ai-projects\lib\site-packages\sklearn\model_selection\_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [7]:
estimator = KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100,batch_size=50, verbose=0)

In [8]:
results = cross_val_score(estimator, features, target, cv=kfold)
print("simple_shallow_seq_model: (%.2f) MSE" % (results.std()))

simple_shallow_seq_model: (162.78) MSE


In [9]:
estimator.fit(features, target)
estimator.model.save('simple_shallow_seq_net.h5')

In [10]:
estimators = []
estimators.append(('standardize',StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=simple_shallow_seq_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [11]:
results = cross_val_score(pipeline, features, target, cv=kfold)
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(67.04) MSE


In [12]:
pipeline.fit(features, target)
pipeline.named_steps['estimator'].model.save('standardised_shallow_seq_net.h5')

In [13]:
def deep_seq_net():
    #create a deep sequential model
    model = Sequential()
    model.add(Dense(7, input_dim=7, kernel_initializer='normal', activation='sigmoid'))
    model.add(Dense(7, activation='tanh'))
    model.add(Dense(7, activation='sigmoid'))
    model.add(Dense(7, activation='tanh'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [14]:
#Standardize the variables in the training data and then fit the shallow neural net model to the training data
#Create the pipeline and fit the model using standardized data
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_seq_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [15]:
#Cross-validate the fit model across the subsets of the data and print the MSE
results = cross_val_score(pipeline, features, target, cv=kfold)
print("simple_std_shallow_seq_net:(%.2f) MSE" % (results.std()))

simple_std_shallow_seq_net:(60.32) MSE


In [16]:
pipeline.fit(features, target)
pipeline.named_steps['estimator'].model.save('deep_seq_net.h5')

In [17]:
#Widen the neural network by increasing number of neurons(nodes) in each layer
def deep_and_wide_net():
    #create a sequential model
    model = Sequential()
    model.add(Dense(21, input_dim=7, kernel_initializer='normal', activation='relu'))
    model.add(Dense(21, activation='relu'))
    model.add(Dense(21, activation='relu'))
    model.add(Dense(21, activation='sigmoid'))
    model.add(Dense(1, kernel_initializer='normal'))
    sgd = optimizers.SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd)
    return model

In [18]:
#Standardize the variables in training data and then fit deep and wide neural net model to training data
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('estimator', KerasRegressor(build_fn=deep_and_wide_net, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)

In [19]:
results = cross_val_score(pipeline, features, target, cv=kfold)
print("deep_and_wide_model:(%.2f) MSE" % (results.std()))

deep_and_wide_model:(68.01) MSE


In [20]:
pipeline.fit(features, target)
pipeline.named_steps['estimator'].model.save('deep_and_wide_net.h5')